# Getting the input from the netlist file
In this cell the files are inputted and transfered into a list for easy manipulation.  
A class is defined to take in the value from each line. The edges list is included. Then this list is passed to the inbuilt edge -graph function from the networks library.  
Now the data from the input file is transfered into a list of dictionaries for later access. And another gate type dictionary is created to find the type of the gate from the output terminal name.  
Finally the graph is sorted topologically and saved as nl 

In [2]:
# Importing the necessary libraries
import networkx as nx

# Declaring variables for opening the files required = Pass the name of the necessary files through the variables mentioned below
netwrok = "c17.net"
inputs = "c17.inputs"

# Defining a class to take in the vlaues from the network file
class Component:
	def __init__(self,name,g_type,input,output):
		self.name = name
		self.g_type = g_type
		self.input = input
		self.output = output
                
# Declaring a list to store the components
l = []

# Opening the file mentioned and storing it in the list
with open(netwrok) as f:
    lines = f.readlines()
    for line in lines:
        name,g_type,*input= line.split()
        output = line.split()[-1]
        Gate = Component(name,g_type,input,output)
        l.append(Gate)

# Declaring and creating the  edges list for the topoligal sorting
edges = []
g = nx.DiGraph()
for gate in l:
    if len(gate.input) == 3:
        edges.append((gate.input[0],gate.output))
        edges.append((gate.input[1],gate.output))
    else:
        edges.append((gate.input[0],gate.output))

# Making the diGraph for the topological sorting
g.add_edges_from(edges)

# Taking inputs from the inputs file and storing them in the values list as dictionaries
value = []
with open(inputs) as file:
    lineIn = file.readlines()
    pi = lineIn[0].split()
    for i in range(len(lineIn)-1):
        value.append({})
        for j in range(len(pi)):
            value[i].update({lineIn[0].split()[j]:lineIn[i+1].split()[j]})
value1 = value.copy()

# Saving the gate_types of eadh gate as a dictionary for later use in the gate function
gate_type = {}
for node in pi:
     gate_type.update({node:"PI"})
for node in l:
     gate_type.update({node.output:node.g_type})
nx.set_node_attributes(g,gate_type,name = "gateType")

# Making a topologically sorted list of the nodes
nl = list(nx.topological_sort(g))

# Solving using the topological order
Topological solving works by finding the value of each node from the predeseesors in topological order. The sorted list of nodes are iterated over to find their type and predessors, and then to find the output based on these parameters . All these steps are iterated over the number of input sets given

In [19]:
# Iterating through all the input sets given
for i in range(len(lineIn)-1):   

   # Iterating through the gates except pi
   for j in range(len(pi),len(nl)):

      # Finding the predecessors of the gate
      p = list(g.predecessors(nl[j]))

      out = 0
      
      # Checking for all types of gates and performing the necessary operations
      if gate_type[nl[j]] == "nand2":
         out = int(not(int(value[i][p[0]]) and int(value[i][p[1]])))
            
      elif gate_type[nl[j]] == "and2":
         out = int(int(value[i][p[0]]) and int(value[i][p[1]]))

      elif gate_type[nl[j]] == "nor2":
         out = int(not(int(value[i][p[0]]) or int(value[i][p[1]])))
      
      elif gate_type[nl[j]] == "or2":
         out = int(int(value[i][p[0]]) or int(value[i][p[1]]))

      elif gate_type[nl[j]] == "inv":
         out = int(not(int(value[i][p[0]])))

      elif gate_type[nl[j]] == "xnor2":
         out = int(not(int(value[i][p[0]]) ^ int(value[i][p[1]])))

      elif gate_type[nl[j]] == "xor2":
         out = int(int(value[i][p[0]]) ^ int(value[i][p[1]]))
      
      # Updating the output of the current gate into the value list
      value[i].update({nl[j]:out})

#Sorting the result in alphabetical order
value_sorted = []
for i in range(len(lineIn)-1):
   value_sorted.append(sorted(value[i].items()))

# Printing the result into a file "Output.txt"
with open('Output.txt', 'w') as file:
   for tpl in value_sorted[0]:
         file.write(str(tpl[0]))
         file.write(" ")
   for i in range(len(lineIn)-1):
        file.write("\n")
        for tpl in value_sorted[i]:
            file.write(str(tpl[1]))
            file.write(" ")

# Printing it in the cell
for tpl in value_sorted[0]:
    print(tpl[0], end=" ")
for i in range(len(lineIn)-1):
   print("\n")
   for tpl in value_sorted[i]:
      print(tpl[1], end=" ")

N1 N2 N22 N23 N3 N6 N7 n_0 n_1 n_2 n_3 

0 1 1 1 0 0 0 1 1 1 0 

0 0 0 0 1 0 0 1 1 1 1 

1 0 0 0 0 0 0 1 1 1 1 

0 0 0 0 1 1 1 1 0 1 1 

1 1 1 0 1 1 1 0 0 1 1 

1 1 1 1 1 0 0 0 1 1 0 

1 1 1 0 1 1 0 0 0 1 1 

1 1 1 1 0 0 0 1 1 1 0 

0 1 1 1 1 0 1 1 1 0 0 

0 0 0 0 1 1 0 1 0 1 1 

# Using the Event - driven method
Event-driven method works by updating the states in an event driven approach. This utilises a queue that contuially gets update by popping elements and adding its sucessors, given that both its predesessors are present in the value list already. The popped value if not already in the value list , is then added by considering the type of gate and the predessors. All these steps are iterated over the number of input sets given

In [18]:
# Importing the
from queue import Queue

# Declaring and initializing the queue
Event = Queue()
for item in nl[:len(pi)]:
    Event.put(item)

# Iterating through all the input sets given
for i in range(len(lineIn)-1):
    

    while not Event.empty():
        e = Event.get()
        # Adding the succesors of the thing to the queue only is all the predessors are there in the value list already
        for thing in list(g.successors(e)):
            if all(elem in value1[i] for elem in list(g.predecessors(thing))):
                Event.put(thing)

        # Checking if the value of the key already exist
        if e not in value1[i]:
            print(e)
            p = list(g.predecessors(e))
            out = 0
            
            # Checking and executing all the gates
            if gate_type[e] == "nand2":
                out = int(not(int(value[i][p[0]]) and int(value[i][p[1]])))
                    
            elif gate_type[e] == "and2":
                out = int(int(value[i][p[0]]) and int(value[i][p[1]]))

            elif gate_type[e] == "nor2":
                out = int(not(int(value[i][p[0]]) or int(value[i][p[1]])))
            
            elif gate_type[e] == "or2":
                out = int(int(value[i][p[0]]) or int(value[i][p[1]]))

            elif gate_type[e] == "inv":
                out = int(not(int(value[i][p[0]])))

            elif gate_type[e] == "xnor2":
                out = int(not(int(value[i][p[0]]) ^ int(value[i][p[1]])))

            elif gate_type[e] == "xor2":
                out = int(int(value[i][p[0]]) ^ int(value[i][p[1]]))
            
            # Adding the key value pair to the value list
            value1[i].update({e:out})

#Sorting the result in alphabetical order            
value_sorted = []
for i in range(len(lineIn)-1):
   value_sorted.append(sorted(value1[i].items()))

# Printing the result into a file "Output.txt"
with open('Output.txt', 'w') as file:
   for tpl in value_sorted[0]:
         file.write(str(tpl[0]))
         file.write(" ")
   for i in range(len(lineIn)-1):
        file.write("\n")
        for tpl in value_sorted[i]:
            file.write(str(tpl[1]))
            file.write(" ")

# Printing it in the cell
for tpl in value_sorted[0]:
    print(tpl[0], end=" ")
for i in range(len(lineIn)-1):
   print("\n")
   for tpl in value_sorted[i]:
      print(tpl[1], end=" ")


N1 N2 N22 N23 N3 N6 N7 n_0 n_1 n_2 n_3 

0 1 1 1 0 0 0 1 1 1 0 

0 0 0 0 1 0 0 1 1 1 1 

1 0 0 0 0 0 0 1 1 1 1 

0 0 0 0 1 1 1 1 0 1 1 

1 1 1 0 1 1 1 0 0 1 1 

1 1 1 1 1 0 0 0 1 1 0 

1 1 1 0 1 1 0 0 0 1 1 

1 1 1 1 0 0 0 1 1 1 0 

0 1 1 1 1 0 1 1 1 0 0 

0 0 0 0 1 1 0 1 0 1 1 

In [16]:
for i in range(len(lineIn)-1):
    if value[i] != value1[i]:
        print("Some difference is there")